In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import sys
sys.path.append('..')
import misc.autointerpolation

# Reload misc module after editing
import importlib
importlib.reload(misc.autointerpolation)
importlib.reload(misc)

<module 'misc' (namespace)>

# 1. Load and Preprocess Data <a class="anchor" id="load_data"></a>

In [2]:
path = "E:/Projects/SEACAR_WQ_Pilot/"
gis_path = path+'GIS_data/'

# path = "D:/Water_Quailty/"
# gis_path = path+'GIS_data/'

Combine and preprocess discrete and continuous data.

In [3]:
# dfDis, dfCon = misc.preprocess.preprocess(path + r"OEAT_Discrete-2023-Jan-23.csv", 
#                     path + r"OEAT_Continuous_1-2023-Jan-23.csv",
#                     path + r"OEAT_Continuous_2-2023-Jan-23.csv")

# dfAll = pd.concat([dfDis,dfCon]).drop(columns = ['Hour'])

# dfAll.to_csv(path + "all_0214.csv")

In [4]:
# Reload modules in external .py files after editing.
dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])
# Convert SampleDate froms str to date
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate']).dt.date

C:\Users\qiangy\AppData\Local\ESRI\conda\envs\geo3\lib\site-packages\IPython\core\interactiveshell.py:3442: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
col_ls = ['RowID','ParameterName','ParameterUnits','ProgramLocationID','ActivityType','ManagedAreaName',
                   'SampleDate','Year','Month','ResultValue','ValueQualifier','Latitude_DD','Longitude_DD']
para_ls = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
para_ls_ab = ["S","TN","DO","T","SD"]
# Convert full MA names to short names
dictArea    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'GTM Reserve','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}

# Convert full MA names to MA name in ORCP_Managed_Areas_Oct2021
dictArea2    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Gasparilla Sound-Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend Seagrasses',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'Guana Tolomato Matanzas NERR','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island Sound'}

# Convert full MA names to abbreviations
dictArea3    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'ch','Big Bend Seagrasses Aquatic Preserve':'bb',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'gtm','Estero Bay Aquatic Preserve':'eb',
                'Biscayne Bay Aquatic Preserve':'bbay','Matlacha Pass Aquatic Preserve':'mp',
                'Lemon Bay Aquatic Preserve':'lb','Cape Haze Aquatic Preserve':'ch','Pine Island Sound Aquatic Preserve':'pi'}

dictPara = {"Salinity":'S','Total Nitrogen':'TN','Dissolved Oxygen':'DO','Turbidity':'T','Secchi Depth':'SD'}
dictUnits   = {"Salinity":"ppt","Total Nitrogen": "mg/L","Dissolved Oxygen": "mg/L","Turbidity": "NTU", "Secchi Depth": "m"}
listArea    = dfAll["ManagedAreaName"].unique()
listPara    = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
SpatialRef = '3086'

# 2. Run spatial interpolation through seasonality table <a class="anchor" id="combine"></a>

Delete incorrect seasons

In [6]:
dfSeason = pd.read_csv(path + "OEATUSF_Geospatial_TempSeasons_update_0517.csv").drop(columns = "Unnamed: 0")
dfSeason = dfSeason.dropna(subset=["s_start","s_end"]).reset_index(drop=True)

dfSeason['s_start'] = pd.to_datetime(dfSeason['s_start'])
dfSeason["s_end"] = pd.to_datetime(dfSeason["s_end"])
dfSeason["date_range"] = dfSeason["s_end"] - dfSeason["s_start"]
dfSeason["date_range"] = dfSeason.apply(lambda x: int(str(x["date_range"]).split(" ")[0]),axis=1)
dfSeason = dfSeason[dfSeason["date_range"]>0]
dfSeason.head()

,Unnamed: 0.1,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,date_range,covariates,Processed,Total Nitrogen,Dissolved Oxygen,Salinity,Secchi Depth,Turbidity
0,147,temp,Estero Bay Aquatic Preserve,2022,Summer,7-Jun,2022-06-07,2022-08-26,80,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
1,71,temp,Biscayne Bay Aquatic Preserve,2022,Summer,1-Jun,2022-06-01,2022-08-26,86,bathymetry+LDI+popden,1,NaN,NaN,NaN,NaN,NaN
2,70,temp,Biscayne Bay Aquatic Preserve,2022,Spring,2-Mar,2022-03-02,2022-05-31,90,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
3,146,temp,Estero Bay Aquatic Preserve,2022,Spring,27-Feb,2022-02-27,2022-06-06,99,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
4,69,temp,Biscayne Bay Aquatic Preserve,2022,Winter,8-Feb,2022-02-08,2022-03-01,21,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...


In [7]:
# Print all managed areas
display(dfSeason['ma'].unique())

# Create empty columns to store paths of exported files
# data = dfSeason.assign(**{'Total Nitrogen': np.nan, 'Dissolved Oxygen': np.nan, 'Salinity': np.nan, 'Secchi Depth': np.nan, 'Turbidity': np.nan})

array(['Estero Bay Aquatic Preserve', 'Biscayne Bay Aquatic Preserve',
       'Guana Tolomato Matanzas National Estuarine Research Reserve',
       'Gasparilla Sound-Charlotte Harbor Aquatic Preserve',
       'Big Bend Seagrasses Aquatic Preserve'], dtype=object)

Run through all seasons

In [ ]:
for i in dfSeason.index:
#for i in range(177, 178):
    # if dfSeason.iloc[i]["Processed"] == 1: continue
    for para in listPara[0:5]:
        print("Now interpolating: Row {}; Area {}; Year {}; Season {}; para {}".format(i,dictArea3[dfSeason.iloc[i]["ma"]], dfSeason.iloc[i]["st_Year"], dfSeason.iloc[i]["season"],para))
        out_raster,out_ga_layer,ga_to_raster = misc.autointerpolation.interpolation_auto(method = "rk",
                                                                                         gis_path  = gis_path,
                                                                                         dataframe = dfAll,
                                                                                         managed_area = dfSeason.iloc[i]["ma"],
                                                                                         Year = dfSeason.iloc[i]["st_Year"],
                                                                                         Season = dfSeason.iloc[i]["season"],
                                                                                         start_date = dfSeason.iloc[i]["s_start"],
                                                                                         end_date = dfSeason.iloc[i]["s_end"],
                                                                                         parameter = para,
                                                                                         covariates = dfSeason.iloc[i]["covariates"].split("+"))
#                                                                                          out_raster = name,
#                                                                                          out_ga_layer = name,
#                                                                                          predict_std_err = name)
        dfSeason.loc[i, para] = out_raster
        dfSeason.to_csv(path + 'output.csv')

Now interpolating: Row 0; Area eb; Year 2022; Season Summer; para Salinity
Start the interpolation with the RK method
Salinity in 2022 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 0; Area eb; Year 2022; Season Summer; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 56.35489463806152 seconds ---
Now interpolating: Row 0; Area eb; Year 2022; Season Summer; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 53.87638735771179 seconds ---
Now interpolating: Row 0; Area eb; Year 2022; Season Summer; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 59.79321217536926 seconds ---
Now interpolating: Row 0; Area eb; Year 2022; Season Summer; para Secchi Depth
Start the interpolation with the RK method
Secchi Depth in 2022 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKR

Start the interpolation with the RK method
Salinity in 2021 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 8; Area eb; Year 2021; Season Summer; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 99.45229530334473 seconds ---
Now interpolating: Row 8; Area eb; Year 2021; Season Summer; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 100.33215022087097 seconds ---
Now interpolating: Row 8; Area eb; Year 2021; Season Summer; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 104.0412106513977 seconds ---
Now interpolating: Row 8; Area eb; Year 2021; Season Summer; para Secchi Depth
Start the interpolation with the RK method
Secchi Depth in 2021 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 9; Area ch; Year 2021; Seaso

Start the interpolation with the RK method
--- Time lapse: 61.08446788787842 seconds ---
Now interpolating: Row 16; Area bbay; Year 2021; Season Winter; para Turbidity
Start the interpolation with the RK method
Turbidity in 2021 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 16; Area bbay; Year 2021; Season Winter; para Secchi Depth
Start the interpolation with the RK method
Secchi Depth in 2021 Winter caused an error:
Failed to execute. Parameters are not valid.
ERROR 000366: Invalid geometry type
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 17; Area bbay; Year 2020; Season Fall; para Salinity
Start the interpolation with the RK method
--- Time lapse: 154.78653264045715 seconds ---
Now interpolating: Row 17; Area bbay; Year 2020; Season Fall; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 348.8550546169281 seconds ---
Now interpolating:

--- Time lapse: 206.17574501037598 seconds ---
Now interpolating: Row 25; Area bbay; Year 2020; Season Summer; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 348.9369766712189 seconds ---
Now interpolating: Row 25; Area bbay; Year 2020; Season Summer; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 324.69954919815063 seconds ---
Now interpolating: Row 25; Area bbay; Year 2020; Season Summer; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 205.81990838050842 seconds ---
Now interpolating: Row 25; Area bbay; Year 2020; Season Summer; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 87.37803435325623 seconds ---
Now interpolating: Row 26; Area bbay; Year 2020; Season Spring; para Salinity
Start the interpolation with the RK method
--- Time lapse: 73.26775860786438 seconds ---
Now interpolating: Row 26; Area bbay; Year 2020; Season Spring; para Total Nitrogen
Start the int

Start the interpolation with the RK method
--- Time lapse: 277.33942794799805 seconds ---
Now interpolating: Row 33; Area bb; Year 2020; Season Winter; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 251.15032601356506 seconds ---
Now interpolating: Row 34; Area bb; Year 2019; Season Fall; para Salinity
Start the interpolation with the RK method
--- Time lapse: 775.8714151382446 seconds ---
Now interpolating: Row 34; Area bb; Year 2019; Season Fall; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 685.6329901218414 seconds ---
Now interpolating: Row 34; Area bb; Year 2019; Season Fall; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 813.1689221858978 seconds ---
Now interpolating: Row 34; Area bb; Year 2019; Season Fall; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 783.8900754451752 seconds ---
Now interpolating: Row 34; Area bb; Year 2019; Season Fall; para Secchi D

...

Start the interpolation with the RK method
Total Nitrogen in 2009 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 176; Area gtm; Year 2009; Season Spring; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 319.7768757343292 seconds ---
Now interpolating: Row 176; Area gtm; Year 2009; Season Spring; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 286.099995136261 seconds ---
Now interpolating: Row 176; Area gtm; Year 2009; Season Spring; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 258.31419467926025 seconds ---
Now interpolating: Row 177; Area bb; Year 2009; Season Spring; para Salinity
Start the interpolation with the RK method
--- Time lapse: 7332.864861726761 seconds ---
Now interpolating: Row 177; Area bb; Year 2009; Season Spring; para Total Nitrogen
Start the interpolation with the RK method
--- Time 

--- Time lapse: 277.3613317012787 seconds ---
Now interpolating: Row 184; Area gtm; Year 2008; Season Summer; para Secchi Depth
Start the interpolation with the RK method
Secchi Depth in 2008 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpolating: Row 185; Area bb; Year 2008; Season Summer; para Salinity
Start the interpolation with the RK method
--- Time lapse: 8940.54977273941 seconds ---
Now interpolating: Row 185; Area bb; Year 2008; Season Summer; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 921.372882604599 seconds ---
Now interpolating: Row 185; Area bb; Year 2008; Season Summer; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 7813.946079015732 seconds ---
Now interpolating: Row 185; Area bb; Year 2008; Season Summer; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 4512.182986736298 seconds ---
Now interpol

Start the interpolation with the RK method
--- Time lapse: 3325.6850900650024 seconds ---
Now interpolating: Row 193; Area bb; Year 2008; Season Winter; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 631.7876417636871 seconds ---
Now interpolating: Row 193; Area bb; Year 2008; Season Winter; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 5025.034651041031 seconds ---
Now interpolating: Row 193; Area bb; Year 2008; Season Winter; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 3070.468280315399 seconds ---
Now interpolating: Row 193; Area bb; Year 2008; Season Winter; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 1367.1429388523102 seconds ---
Now interpolating: Row 194; Area bb; Year 2007; Season Fall; para Salinity
Start the interpolation with the RK method
--- Time lapse: 7646.001187086105 seconds ---
Now interpolating: Row 194; Area bb; Year 2007; Season Fall; p

--- Time lapse: 456.37110209465027 seconds ---
Now interpolating: Row 202; Area ch; Year 2007; Season Spring; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 2700.664624452591 seconds ---
Now interpolating: Row 202; Area ch; Year 2007; Season Spring; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 1005.3963735103607 seconds ---
Now interpolating: Row 202; Area ch; Year 2007; Season Spring; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 2735.9794924259186 seconds ---
Now interpolating: Row 203; Area eb; Year 2007; Season Spring; para Salinity
Start the interpolation with the RK method
--- Time lapse: 272.2640850543976 seconds ---
Now interpolating: Row 203; Area eb; Year 2007; Season Spring; para Total Nitrogen
Start the interpolation with the RK method
--- Time lapse: 115.84159445762634 seconds ---
Now interpolating: Row 203; Area eb; Year 2007; Season Spring; para Dissolved Oxygen
Start the interp

--- Time lapse: 392.81579422950745 seconds ---
Now interpolating: Row 211; Area ch; Year 2006; Season Fall; para Dissolved Oxygen
Start the interpolation with the RK method
--- Time lapse: 3136.330058813095 seconds ---
Now interpolating: Row 211; Area ch; Year 2006; Season Fall; para Turbidity
Start the interpolation with the RK method
--- Time lapse: 780.4624109268188 seconds ---
Now interpolating: Row 211; Area ch; Year 2006; Season Fall; para Secchi Depth
Start the interpolation with the RK method
--- Time lapse: 3201.409134864807 seconds ---
Now interpolating: Row 212; Area gtm; Year 2006; Season Summer; para Salinity
Start the interpolation with the RK method
--- Time lapse: 273.8999032974243 seconds ---
Now interpolating: Row 212; Area gtm; Year 2006; Season Summer; para Total Nitrogen
Start the interpolation with the RK method
Total Nitrogen in 2006 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

Now interpol